In [ ]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import numpy as np
from math import *
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import re

In [ ]:
txt = open('C3S_StormTracks_era5_19792021_0100_v1.dat')
txt = txt.read().split('\n')
point_num_120 = np.empty(120,dtype = 'int')
num = 0
for i in txt:
    if bool(re.match('POINT_NUM',i)) == True:
        point_num_120[num] = int(re.sub('POINT_NUM','',i))
        num += 1

# store tracks in dict, key is ID, value is the array with point_num values, which is an array as well
tracks = {}
ID = 1
start = 2
for row in point_num_120:
    tracks[ID] = np.zeros([row,19])
    for i in range(row):
        col = 0
        for value in np.float_((re.sub('&','',txt[start+i])).split()):
            if value == np.float_(1e+25):
                value = 0
            tracks[ID][i,col] = value
            col += 1
    ID += 1
    start += row+2

In [ ]:
id_near_ireland = [17,18,24,27,47,55,56,58,65,73,102,103]

In [ ]:
lowest_per_ke = np.inf
lowest_ke = np.inf
high_mslp = 0

In [ ]:
store_center_values = []
store_total_energy = []

In [ ]:
# map resolution is different and same resolution in y-axis has the different length

# region lon: -25~7 lat: 47~64
name = ['1988020106-mslp.nc','1988020906-mslp.nc','1989102812-mslp.nc','1990020118-mslp.nc',
        '1995011715-mslp.nc','1997122412-mslp.nc','1998010409-mslp.nc','1998122618-mslp.nc',
        '2000112512-mslp.nc','2005110721-mslp.nc','2013122706-mslp.nc','2014021209-mslp.nc']
name_mslp_v = ['1988_1_2_mslp_v.nc','1988_1_2_mslp_v.nc','1989_10_11_mslp_v.nc','1990_1_2_mslp_v.nc',
              '1995_1_mslp_v.nc','1997_12_mslp_v.nc','1998_1_mslp_v.nc','1998_12_mslp_v.nc',
               '2000_11_mslp_v.nc','2005_11_mslp_v.nc','2013_12_mslp_v.nc','2014_2_mslp_v.nc']
name_3r = ['re_1988_1_2_3r.nc','re_1988_1_2_3r.nc','re_1989_10_11_3r.nc','re_1990_1_2_3r.nc',
           're_1995_1_3r.nc','re_1997_12_3r.nc','re_1998_1_3r.nc',"re_1998_12_3r.nc",
           're_2000_11_3r.nc','re_2005_11_3r.nc','re_2013_12_3r.nc','re_2014_2_3r.nc']

for i in range(0,12):
    filename = name[i]
    mslp = xr.open_dataset(filename)
    mslp = mslp.to_array()[0,:,:]/100
    
    
    # plot
    fig = plt.figure(figsize = [6,8])
    ax = plt.axes(projection = ccrs.Mercator())
    ax.coastlines(resolution = '50m', color ="black",linewidth = 1.)
    ax.set_extent([-75., 20., 35., 75.])
    cp = ax.contour(mslp.lon, mslp.lat, mslp,
                       np.arange(940, 1050, 5), cmap='jet', transform=ccrs.PlateCarree())
    plt.clabel(cp,inline=1,fontsize=12,fmt='%1.0f',inline_spacing=1, colors='black')
    ax.add_feature(cfeature.LAND)
    grid = ax.gridlines(draw_labels=True,linestyle = "--")

    # track
    t = float(filename.split("-")[0])
    id_in_tracks = [index for index in id_near_ireland if t in tracks[index]][0]
    lon = []
    lat = []
    mlon = 0
    mlat = 0
    mlon_index = 0
    mlat_index = 0
    centra_mslp = 1100 
    for index,element in np.ndenumerate(tracks[id_in_tracks]):
        if index[1] == 4:
            if tracks[id_in_tracks][index] > 360 or tracks[id_in_tracks][index]<=0 or tracks[id_in_tracks][(index[0],index[1]+1)] <= 0 or tracks[id_in_tracks][(index[0],index[1]+1)]>=90:
                continue
            else:
                lon.append(element if element<180 else element-360)
                lat.append(tracks[id_in_tracks][(index[0],index[1]+1)])
                if tracks[id_in_tracks][(index[0],index[1]+2)] < centra_mslp:
                    centra_mslp = tracks[id_in_tracks][(index[0],index[1]+2)]
                    mlon = element
                    mlat = tracks[id_in_tracks][(index[0],index[1]+1)]

    plt.plot(lon,lat,marker='o',color = "green",markersize = 5,transform = ccrs.PlateCarree())
    plt.plot([mlon],[mlat],marker = 'o',markersize = 8,
                color = "red",transform = ccrs.PlateCarree())

    # boundary and region
    mlon = int(mlon) if mlon- int(mlon) <0.5 else ceil(mlon)
    mlat = int(mlat) if mlat- int(mlat)<0.5 else ceil(mlat)
    # boundary: red box
    # region : green box
    boundary = [];region = []
    n_lat = 0;n_lon = 0
    coor_index = []
    coor = []
    mcoor = []
    # loop from lower-left cornor to upper-right cornor
    for lat in mslp.lat.values:
        n_lon = 0
        for lon in mslp.lon.values:
            if lat == mlat and lon == mlon-360:
                mcoor.append((180-n_lat,n_lon))
            if (47 <= lat <= 64) and (-25<=lon<=7):
                coor_index.append((180-n_lat,n_lon))
                coor.append((lat,lon))
            if ((lat == 47 or lat == 64) and (-25<=lon<=7)) or ((lon == -25 or lon==7) and (47<=lat<=64)):
                region.append((lat,lon))
            if (lat == mlat+7 or lat == mlat-5) and (lon <= mlon+12-360 and lon >= mlon-12-360):
                boundary.append((lat,lon))
            if (lon == mlon+12-360 or lon == mlon-12-360) and (lat <= mlat+7 and lat >= mlat-5):
                boundary.append((lat,lon))
            n_lon += 1
        n_lat+=1
    plt.scatter([value[1] for value in region],[value[0] for value in region],
               marker = 'o',s= 4,color = "green",transform = ccrs.PlateCarree())

    plt.scatter([value[1] for value in boundary],[value[0] for value in boundary],
               marker = 'o',s= 4,color = "red",transform = ccrs.PlateCarree())
    plt.title("mslp at "+name[i].split("-")[0])
    
    plt.show()
    
    

    g = 9.80665

    filename = name_3r[i]
    data = xr.open_dataset(filename)

    # energy of different volumes
    energy = []

    # calculate the energy per geopotential height
    energy_per_metre = []

    # wind speed
    ws = np.sqrt(data.v**2+data.u**2).values

    # height m/s
    height = (data.z/g).values

    # geopotential m^2s^-2
    geo = data.z.values

    # area of each grid in the fixed rectangle
    # km^2
    area = []
    for axes in coor:
        # km
        r = cos((axes[0]/180)*np.pi)*6371
        c = 2*np.pi*r
        area.append(c/(4*360)*(2*np.pi*6371/360/4))

    # calculate the energy of different vertical volumes of each grid at different time
    n_time = 0
    for time in data.time.values:
        e_time = []
        e_time_per_metre = []
        for plev in range(len(data.level.values)-1):
            # hPa
            diff_plev = data.level.values[3-plev] - data.level.values[3-(plev+1)]
            e_plev = []
            e_plev_per_meter = []
            num = 0
            for axes in coor_index:
                # km^2 * m
                volume = area[num]*(height[n_time,3-plev-1,axes[0],axes[1]]-height[n_time,3-plev,axes[0],axes[1]])
                # hPa * s^2 * m^-2
                density = diff_plev/(geo[n_time,3-plev-1,axes[0],axes[1]]-geo[n_time,3-plev,axes[0],axes[1]])
                # km^2 * hPa * m  = km^2 * hkg * s^-2 
                e = 1/2*volume*density*(((ws[n_time,3-plev,axes[0],axes[1]]+ws[n_time,3-plev-1,axes[0],axes[1]])/2)**2)
                e_plev.append(e/10**11)
                e_plev_per_meter.append(
                    (e/(height[n_time,3-plev-1,axes[0],axes[1]]-height[n_time,3-plev,axes[0],axes[1]]))/(10**3))
                num += 1
            e_time.append(e_plev)
            e_time_per_metre.append(e_plev_per_meter)
        
        tot_per_metre = []
        tot = []
        num = 0
        for axes in coor_index:
            volume = area[num]*(height[n_time,0,axes[0],axes[1]]-height[n_time,3,axes[0],axes[1]])
            density = (925-250)/(geo[n_time,0,axes[0],axes[1]]-geo[n_time,3,axes[0],axes[1]])
            e = 1/2*volume*density*(((ws[n_time,0,axes[0],axes[1]]+ws[n_time,3,axes[0],axes[1]])/2)**2)
            tot.append(e/10**11)
            tot_per_metre.append((e/(height[n_time,0,axes[0],axes[1]]-height[n_time,3,axes[0],axes[1]]))/10**3)
            num+=1
        e_time.append(tot)
        e_time_per_metre.append(tot_per_metre)
        
        energy.append(e_time)
        energy_per_metre.append(e_time_per_metre)
        n_time += 1

    print("----------------------------------------------------------------------------------------------------")
    

    filename = name_mslp_v[i]
    data_mslp = xr.open_dataset(filename).msl/100

    # calculate total energy of storm at each time
    e_925_750 = []
    e_750_500 = []
    e_500_250 = []
    e_tot = []

    # per metre
    e_925_750_per_metre = []
    e_750_500_per_metre = []
    e_500_250_per_metre = []
    e_925_250_per_metre = []

    for time in range(len(data.time.values)):
        e_925_750.append(sum(energy[time][0]))
        e_925_750_per_metre.append(sum(energy_per_metre[time][0])/len(energy_per_metre[time][0]))
        e_750_500.append(sum(energy[time][1]))
        e_750_500_per_metre.append(sum(energy_per_metre[time][1])/len(energy_per_metre[time][1]))
        e_500_250.append(sum(energy[time][2]))
        e_500_250_per_metre.append(sum(energy_per_metre[time][2])/len(energy_per_metre[time][2]))
        e_tot.append(np.sum(energy[time][0])+np.sum(energy[time][1])+np.sum(energy[time][2]))
        e_925_250_per_metre.append(sum(energy_per_metre[time][3])/len(energy_per_metre[time][3]))

         
        
    plt.figure(figsize = (15,6))
    plt.plot(data.time,e_925_750,color = "c",label = "925hPa ~ 750hPa")
    plt.plot(data.time,e_750_500,color = "r",label = "750hPa ~ 500hPa")
    plt.plot(data.time,e_500_250,color = "g",label = "500hPa ~ 250hPa")
    plt.plot(data.time,e_tot,color = "black",label = "total")
    plt.ylabel("total energy (10^8  J)")
    plt.legend(loc = 'upper left')

    # find center
    m_time = list(str(t)[:-2])
    m_time.insert(4,"-");m_time.insert(7,"-");m_time.insert(10,"T")
    m_time = "".join(m_time)
    num = 0
    for time in data.time.values:
        if str(time)[0:13] == m_time: break
        num+=1
    plt.axvline(data.time.values[num],linestyle = 'dashed')

    # add mslp at the center of storm
    axis2 = plt.twinx()
    axis2.plot(data_mslp.time,data_mslp[:,mcoor[0][0],mcoor[0][1]],color = "orange",label = "mslp")
    axis2.set_ylabel('mslp')

    # calculate the averaged mslp in the fixed green box
    averaged_mslp = []
    for time in range(len(data_mslp.time.values)):
        value = 0
        for axes in coor_index:
            value += data_mslp.values[time,axes[0],axes[1]]
        averaged_mslp.append(value/len(coor_index))
    axis2.plot(data_mslp.time,averaged_mslp,color = "blue",label = "averaged mslp")
    plt.legend(loc = 'upper right')
    plt.title("Total energy in different depth of layers at "+name[i].split("-")[0])
    plt.show() 
    
    print("----------------------------------------------------------------------------------------------------")

    plt.figure(figsize = (15,6))
    plt.plot(data.time,e_925_750_per_metre,color = "c",label = "925hPa ~ 750hPa")
    plt.plot(data.time,e_750_500_per_metre,color = "r",label = "750hPa ~ 500hPa")
    plt.plot(data.time,e_500_250_per_metre,color = "g",label = "500hPa ~ 250hPa")
    plt.plot(data.time,e_925_250_per_metre,color = 'black',label = "925hPa ~ 250hPa")
    plt.ylabel("total energy (10^8  J)")
    plt.legend(loc = 'upper left')
    # center
    plt.axvline(data.time.values[num],linestyle = 'dashed')

    axis2 = plt.twinx()
    axis2.plot(data_mslp.time,data_mslp[:,mcoor[0][0],mcoor[0][1]],color = "orange",label = "mslp")
    axis2.set_ylabel('mslp')

    axis2.plot(data_mslp.time,averaged_mslp,color = "blue",label = "averaged mslp")
    plt.legend(loc = 'upper right')
    plt.title("Energy per geopotential metre in different depth of layers at "+name[i].split("-")[0])

    plt.show()
    
    print("----------------------------------------------------------------------------------------------------")

    # start_time
    s_t = list(str(tracks[id_in_tracks][0][0])[:-2])
    s_t.insert(4,"-");s_t.insert(7,"-");s_t.insert(10,"T")
    s_t = "".join(s_t)
    # end_time
    e_t = list(str(tracks[id_in_tracks][-1][0])[:-2])
    e_t.insert(4,"-");e_t.insert(7,"-");e_t.insert(10,"T")
    e_t = "".join(e_t)

    s_index = 0
    e_index = 0
    number = 0
    for time in data.time.values:
        if str(time)[0:13] == s_t: s_index = number
        if str(time)[0:13] == e_t: e_index = number
        number+=1
    
    for time in range(s_index,e_index+1):
        store_total_energy.append(e_tot[time])
    
    #plot
    plt.figure(figsize = (12,7),dpi = 500)
    plt.axvline(data.time.values[num],linestyle = 'dashed')
    plt.text(data.time.values[num],e_tot[num]/2,m_time)

    plt.plot(data.time[s_index:e_index+1],e_925_750[s_index:e_index+1],color = "c",label = "925hPa ~ 750hPa")
    plt.text(data.time[num],e_925_750[num],round(e_925_750[num],3))
    plt.plot(data.time[s_index:e_index+1],e_750_500[s_index:e_index+1],color = "r",label = "750hPa ~ 500hPa")
    plt.plot(data.time[s_index:e_index+1],e_500_250[s_index:e_index+1],color = "g",label = "500hPa ~ 250hPa")
    plt.plot(data.time[s_index:e_index+1],e_tot[s_index:e_index+1],color = "black",label = "925hPa ~ 250hPa")
    plt.ylabel("Total energy in the region (unit : $10^{19}$ J)")
    plt.legend(loc = 'upper left')

    axis2 = plt.twinx()
    axis2.plot(data_mslp.time[s_index:e_index+1],data_mslp[:,mcoor[0][0],mcoor[0][1]][s_index:e_index+1],
               color = "orange",label = "MSLP at MMSLP Postion")
    axis2.set_ylabel('Pressure (unit : hPa)')
    axis2.plot(data_mslp.time[s_index:e_index+1],averaged_mslp[s_index:e_index+1],
               color ="blue",label = "Average MSLP in Fixed Region")
    plt.legend(loc = 'upper right')
    plt.title("Energy from "+str(data.time.values[s_index])[0:13]+" to "+str(data.time.values[e_index])[0:13])
    plt.show() 
    
    print("----------------------------------------------------------------------------------------------------")
    #plot
    plt.figure(figsize = (12,7),dpi = 500)
    plt.axvline(data.time.values[num],linestyle = 'dashed')
    plt.text(data.time.values[num],e_500_250_per_metre[num],m_time)

    plt.plot(data.time[s_index:e_index+1],e_925_750_per_metre[s_index:e_index+1],color = "c",label = "925hPa ~ 750hPa")
    plt.text(data.time[num],e_925_750_per_metre[num],round(e_925_750_per_metre[num],3))
    plt.plot(data.time[s_index:e_index+1],e_750_500_per_metre[s_index:e_index+1],color = "r",label = "750hPa ~ 500hPa")
    plt.plot(data.time[s_index:e_index+1],e_500_250_per_metre[s_index:e_index+1],color = "g",label = "500hPa ~ 250hPa")
    plt.plot(data.time[s_index:e_index+1],e_925_250_per_metre[s_index:e_index+1],color = "black",label = "925hPa ~ 250hPa")
    plt.ylabel("Gird-scale power (unit : $10^{11}$ J)")
    plt.legend(loc = 'upper left')

    axis2 = plt.twinx()
    axis2.plot(data_mslp.time[s_index:e_index+1],data_mslp[:,mcoor[0][0],mcoor[0][1]][s_index:e_index+1],
               color = "orange",label = "MSLP at MMSLP Postion")
    axis2.set_ylabel('Pressure (unit : hPa)')
    axis2.plot(data_mslp.time[s_index:e_index+1],averaged_mslp[s_index:e_index+1],
               color ="blue",label = "Average MSLP in Fixed Region")
    plt.legend(loc = 'upper right')
    plt.title("Energy per Geopotential Meter from "+str(data.time.values[s_index])[0:13]+" to "+str(data.time.values[e_index])[0:13])
    plt.show() 
    
    
    
    if e_925_750_per_metre[num] < lowest_per_ke:
        lowest_per_ke = e_925_750_per_metre[num]
    
    if averaged_mslp[num] > high_mslp:
        high_mslp = averaged_mslp[num]
    
    if e_925_750[num] < lowest_ke:
        lowest_ke = e_925_750[num]
    print(i)
    print("total energy:")
    print(e_tot[num])
    print("energy per geopotential height:")
    print(e_925_250_per_metre[num])
    
    store_center_values.append(
        [e_925_750_per_metre[num],e_750_500_per_metre[num],e_500_250_per_metre[num],e_925_250_per_metre[num]])

In [ ]:
# check 1988-02-28 0:00 anti-cyclone with high centre pressure level and high wind speed
# the interaction between cyclone and anti-cyclone may have high wind speed
# single level data
abnormal = ["1988022800-mslp_v.nc","1990020415-mslp_v.nc","1997121612-mslp_v.nc","1998012103-mslp_v.nc",
           "2005112506-mslp_v.nc","2013120509-mslp_v.nc"]

check = xr.open_dataset(abnormal[1])
msl = check.msl[0,:,:]/100
msl

plev_data = xr.open_dataset("1990020415_plev_v.nc")
ws = np.sqrt(plev_data.u**2+plev_data.v**2)

speed = np.sqrt(check.v100**2+check.u100**2)

fig = plt.figure(figsize = [13,13],dpi = 450)
ax = plt.axes(projection = ccrs.Mercator())

ax.set_extent([-75., 20., 35., 75.])
cp = ax.contour(msl.longitude, msl.latitude, msl,
                   np.arange(940, 1020, 5), cmap='jet', transform=ccrs.PlateCarree())

pc = ax.contourf(speed.longitude, speed.latitude, (ws[0,3,:,:]+ws[0,2,:,:])/2, 
                   np.arange(0, 50, 5), cmap='jet', transform=ccrs.PlateCarree())
plt.colorbar(pc, shrink=0.7, label='wind speed m/s')

plt.clabel(cp,inline=1,fontsize=12,fmt='%1.0f',inline_spacing=1, colors='black')

grid = ax.gridlines(draw_labels=True,linestyle = "--")

plt.scatter([value[1] for value in region],[value[0] for value in region],
           marker = 'o',s= 4,color = "green",transform = ccrs.PlateCarree())

ax.coastlines(resolution = '50m', color ="black",linewidth = 1.)
#ax.add_feature(cfeature.LAND)
plt.show()